In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다."),
        ("human", "반가워요! 제 이름은 {name} 입니다."),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{input}"),
    ]
)

prompt.invoke({"name":"박진호", "input":"내 이름으로 삼행시를 지어주세요."}).messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다.'),
 HumanMessage(content='반가워요! 제 이름은 박진호 입니다.'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='내 이름으로 삼행시를 지어주세요.')]

In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown

llm = ChatOpenAI(model='gpt-4o')
chain = prompt | llm | StrOutputParser()

display(Markdown(chain.invoke({"name":"박진호", "input":"내 이름으로 삼행시를 지어주세요."})))

물론이죠, 박진호님의 이름으로 삼행시를 지어볼게요.

**박**: 박진호님의 하루는  
**진**: 진심이 가득한 멋진 날들이며  
**호**: 호기심으로 가득 찬 행복한 시간이 되길 바랍니다.

In [34]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다."),
        ("human", "반가워요! 제 이름은 {name} 입니다."),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

prompt.invoke({
  "name":"박진호", 
  "input":"내 이름으로 날씨에 어울리는 삼행시를 해줘",
  "history":[
    ("human", "오늘의 날씨를 알려줘."),
    ("ai","오늘의 날씨는 '흐림'입니다.")
    ]
  }).messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다.'),
 HumanMessage(content='반가워요! 제 이름은 박진호 입니다.'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='오늘의 날씨를 알려줘.'),
 AIMessage(content="오늘의 날씨는 '흐림'입니다."),
 HumanMessage(content='내 이름으로 날씨에 어울리는 삼행시를 해줘')]

In [36]:
llm = ChatOpenAI(model='gpt-4o')
chain = prompt | llm | StrOutputParser()
display(Markdown(chain.invoke({
  "name":"박진호", 
  "input":"내 이름으로 날씨에 어울리는 삼행시를 해줘",
  "history":[
    ("human", "오늘의 날씨를 알려줘."),
    ("ai","오늘의 날씨는 '흐림'입니다.")
    ]
  })))

물론이죠, 박진호님의 이름으로 삼행시를 지어드릴게요!

박: 박진호님, 오늘 하늘이
진: 진짜 흐리네요.
호: 호우가 내릴 수도 있으니 우산 꼭 챙기세요.

마음에 드셨으면 좋겠네요!

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

llm = ChatOpenAI(model='gpt-4o')

chain = prompt | llm

In [48]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    if session_ids not in store: store[session_ids] = ChatMessageHistory()
    return store[session_ids]

with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        chain,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [49]:
output = with_message_history.invoke({
    "ability": "math",
    "input": "tell me about Navier-Stokes equations shortly"
},
config={"configurable": {"session_id": "aa33dd"}})
display(Markdown(output.content))

Certainly! The Navier-Stokes equations are a set of partial differential equations that describe the motion of fluid substances such as liquids and gases. They are fundamental to the field of fluid dynamics and are used to model weather patterns, ocean currents, water flow in pipes, air flow around wings, and many other phenomena involving fluid flow.

The equations are based on Newton's second law of motion and account for various forces acting on a fluid element, including:

1. **Viscous forces**: These are internal frictional forces within the fluid due to its viscosity.
2. **Pressure forces**: These arise from pressure differences within the fluid.
3. **External forces**: These can include gravity, electromagnetic forces, etc.

The general form of the Navier-Stokes equations for an incompressible fluid is:

\[ \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f} \]

where:
- \(\mathbf{u}\) is the velocity field of the fluid.
- \(t\) is time.
- \(p\) is the pressure field.
- \(\nu\) is the kinematic viscosity of the fluid.
- \(\mathbf{f}\) represents external forces (e.g., gravity).

These equations are accompanied by the continuity equation for incompressible flow:

\[ \nabla \cdot \mathbf{u} = 0 \]

Solving the Navier-Stokes equations can be quite challenging, especially for turbulent flows. Despite their complexity, these equations are essential for understanding and predicting fluid behavior in many practical applications.

In [50]:
translated_output = with_message_history.invoke({
    "ability": "math",
    "input": "translate the previous answer into Korean."
},
config={"configurable": {"session_id": "aa33dd"}})
display(Markdown(translated_output.content))

물론입니다! 나비에-스토크스 방정식(Navier-Stokes equations)은 액체나 기체와 같은 유체 물질의 운동을 설명하는 편미분 방정식들의 집합입니다. 이 방정식들은 유체 역학의 기본이 되며, 날씨 패턴, 해류, 파이프 속의 물 흐름, 날개 주변의 공기 흐름 등 유체 흐름과 관련된 많은 현상을 모델링하는 데 사용됩니다.

이 방정식들은 뉴턴의 제2 운동 법칙을 기반으로 하며, 유체 요소에 작용하는 여러 힘들을 설명합니다. 이러한 힘들은 다음과 같습니다:

1. **점성력**: 점성 때문에 유체 내부의 마찰력입니다.
2. **압력력**: 유체 내부의 압력 차이로 인해 발생하는 힘입니다.
3. **외부 힘**: 중력, 전자기력 등 외부에서 작용하는 힘입니다.

비압축성 유체에 대한 나비에-스토크스 방정식의 일반적인 형태는 다음과 같습니다:

\[ \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f} \]

여기서:
- \(\mathbf{u}\)는 유체의 속도장입니다.
- \(t\)는 시간입니다.
- \(p\)는 압력장입니다.
- \(\nu\)는 유체의 운동 점도입니다.
- \(\mathbf{f}\)는 외부 힘을 나타냅니다 (예: 중력).

이 방정식들은 비압축성 흐름을 위한 연속 방정식과 함께 사용됩니다:

\[ \nabla \cdot \mathbf{u} = 0 \]

나비에-스토크스 방정식을 푸는 것은 특히 난류 흐름의 경우 매우 어려울 수 있습니다. 그 복잡성에도 불구하고, 이 방정식들은 많은 실용적인 응용에서 유체의 행동을 이해하고 예측하는 데 필수적입니다.

In [60]:
list(map(lambda x:x.content[:50], store['aa33dd'].messages))

['tell me about Navier-Stokes equations shortly',
 'Certainly! The Navier-Stokes equations are a set o',
 'translate the previous answer into Korean.',
 '물론입니다! 나비에-스토크스 방정식(Navier-Stokes equations)은 액체나 ']